### Домашнее задание №5 ###
<ol>
    <li>Попробуйте обучить нейронную сеть LSTM на любом другом датасете (любимый временной ряд, текст на русском (другом языке) как генератор или классификатор, или прилагаемый набор airline-passengers - пасажиропоток для авиалиний).
    <li>Опишите, какой результата вы получили? Что помогло вам улучшить ее точность?

In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [2]:

# чтение из файла с текстом 
text =''
with open("yevgeniy-onegin.txt", 'rb') as f:
    lines = []
    for line in f:
        line = line.strip().lower().decode("utf8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)

text = " ".join(lines)
chars = set([c for c in text])
n_chars = len(chars)

In [3]:
n_chars

98

In [4]:
text[0:2000]

'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах he мысля гордый свет забавить, Вниманье дружбы возлюбя, Хотел бы я тебе представить Залог достойнее тебя, Достойнее души прекрасной, Святой исполненной мечты, Поэзии живой и ясной, Высоких дум и простоты; Но так и быть - рукой пристрастной Прими собранье пестрых глав, Полусмешных, полупечальных, Простонародных, идеальных, Небрежный плод моих забав, Бессонниц, легких вдохновений, Незрелых и увядших лет, Ума холодных наблюдений И сердца горестных замет. И жить торопится, и чувствовать спешит. Князь Вяземский "Мой дядя самых честных правил, Когда не в шутку занемог, Он уважать себя заставил И лучше выдумать не мог. Его пример другим наука; Но, боже мой, какая скука С больным сидеть и день и ночь, Не отходя ни шагу прочь! Какое низкое коварство Полуживого забавлять, Ему подушки поправлять, Печально подносить лекарство, Вздыхать и думать про себя: Когда же черт возьмет тебя!" Так думал молодой повеса, Летя в пыли на почтовых, Всевышн

In [5]:
s=''
for i in chars:
    s+=i
print(s)

УoqмИцcшдeП-pЦб€ лТ;яуzоыФfъскХ)пёШ^гbНаshЕэЗwuщфГЛьr(.СnРвvчaтДю"gО!?li'иmЯdжйЖнЭyрkЧЮМКtБ:,АехзВ


In [6]:
# создание индекса символов и reverse mapping
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [7]:
# для удобства выберете фиксированную длину последовательность 10 символов 
maxlen, step = 28, 1
input_chars, label_chars = [], []

# конвертация data в серии разных maxlen-length последовательностей
for i in range(0, len(text) - maxlen, step):
    input_chars.append(text[i: i + maxlen])
    label_chars.append(text[i + maxlen])

In [8]:
input_chars

['Александр Сергеевич Пушкин Е',
 'лександр Сергеевич Пушкин Ев',
 'ександр Сергеевич Пушкин Евг',
 'ксандр Сергеевич Пушкин Евге',
 'сандр Сергеевич Пушкин Евген',
 'андр Сергеевич Пушкин Евгени',
 'ндр Сергеевич Пушкин Евгений',
 'др Сергеевич Пушкин Евгений ',
 'р Сергеевич Пушкин Евгений О',
 ' Сергеевич Пушкин Евгений Он',
 'Сергеевич Пушкин Евгений Оне',
 'ергеевич Пушкин Евгений Онег',
 'ргеевич Пушкин Евгений Онеги',
 'геевич Пушкин Евгений Онегин',
 'еевич Пушкин Евгений Онегин ',
 'евич Пушкин Евгений Онегин Р',
 'вич Пушкин Евгений Онегин Ро',
 'ич Пушкин Евгений Онегин Ром',
 'ч Пушкин Евгений Онегин Рома',
 ' Пушкин Евгений Онегин Роман',
 'Пушкин Евгений Онегин Роман ',
 'ушкин Евгений Онегин Роман в',
 'шкин Евгений Онегин Роман в ',
 'кин Евгений Онегин Роман в с',
 'ин Евгений Онегин Роман в ст',
 'н Евгений Онегин Роман в сти',
 ' Евгений Онегин Роман в стих',
 'Евгений Онегин Роман в стиха',
 'вгений Онегин Роман в стихах',
 'гений Онегин Роман в стихах ',
 'ений Оне

In [9]:
label_chars[:12]

['в', 'г', 'е', 'н', 'и', 'й', ' ', 'О', 'н', 'е', 'г', 'и']

In [10]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), maxlen, n_chars))
y = np.zeros((len(input_chars), n_chars))

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [11]:
char2index['я'], char2index['з']

(20, 96)

In [12]:
X.shape,X[0, :2]

((139881, 28, 98),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]]))

In [13]:
# установка ряда метапареметров для нейронной сети и процесса тренировки
batch_size = 128
hidden_size = 64 #128
num_iterations = 25 # 25 должно быть достаточно
num_epochs = 1
num_preds = 100

In [14]:
model = Sequential()
model.add(
    GRU(
        hidden_size,
        return_sequences=False,
        input_shape=(maxlen, n_chars),
        unroll=True
    )
)
model.add(Dense(n_chars, activation='softmax'))
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop")

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 64)                31296     
                                                                 
 dense (Dense)               (None, 98)                6370      
                                                                 
Total params: 37,666
Trainable params: 37,666
Non-trainable params: 0
_________________________________________________________________


In [15]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(num_iterations):
    # для каждой итерации запуск передачи данных в модель 
    print()
    print("=" * 50)
    print(f'Итерация #: {iteration}')
    model.fit(X, y,
              batch_size=batch_size,
              epochs=num_epochs)

    # Select a random example input sequence
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print(f'Генерация из [ {test_chars} ]')
    print(test_chars, end="")
    for i in range(num_preds):

        # здесь one-hot encoding.
        X_test = np.zeros((1, maxlen, n_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred


Итерация #: 0
1093/1093 [==============================] - 36s 22ms/step - loss: 2.8648
Генерация из [ ей некогда; боится, Медведя  ]
ей некогда; боится, Медведя на на на на на на на на на на на на на на на на на на на на на на на на на на на на на на на на на н
Итерация #: 1


MemoryError: Unable to allocate 1.43 GiB for an array with shape (139881, 28, 98) and data type float32

In [ ]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]
print(test_idx, test_chars)

In [ ]:
test_chars = 'Свеча горе'

# для числа шагов предсказаний использование текущей тренируемой модели 
# конструирование one-hot encoding для тестирования input и добавление предсказания.
print(f'Генерация из [ {test_chars} ]')
print(test_chars, end="")
for i in range(num_preds):

    # здесь one-hot encoding.
    X_test = np.zeros((1, maxlen, n_chars))
    for j, ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = index2char[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred